In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 데이터 위치

#조류해상도 폴더 => /content/drive/MyDrive/조류해상도

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pip==23.2
!pip install --quiet timm pytorch_lightning==1.7.7 torchmetrics==0.11.1
!pip uninstall -y tensorflow
!pip install tensorflow-cpu

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pytorch_lightning as L
import zipfile, shutil

from glob import glob
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torchvision.transforms import v2 as  transforms
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.transforms import RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter
import timm

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, path_col,  mode='train'):
        self.df = df
        self.path_col = path_col
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.mode == 'train':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            label = row['class']
            data = {
                'image':image,
                'label':label
            }
            return data
        elif self.mode == 'val':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            label = row['class']
            data = {
                'image':image,
                'label':label
            }
            return data
        elif self.mode == 'inference':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            data = {
                'image':image,
            }
            return data

    def train_transform(self, image):
        pass

In [ ]:
class CustomCollateFn:
    def __init__(self, transform, mode):
        self.mode = mode
        self.transform = transform

    def __call__(self, batch):
        if self.mode=='train':
            pixel_values = torch.stack([self.transform(data['image']) for data in batch])
            label = torch.LongTensor([data['label'] for data in batch])
            return {
                'pixel_values':pixel_values,
                'label':label,
            }
        elif self.mode=='val':
            pixel_values = torch.stack([self.transform(data['image']) for data in batch])
            label = torch.LongTensor([data['label'] for data in batch])
            return {
                'pixel_values':pixel_values,
                'label':label,
            }
        elif self.mode=='inference':
            pixel_values = torch.stack([self.transform(data['image']) for data in batch])
            return {
                'pixel_values':pixel_values,
            }


In [ ]:
class CustomModel(nn.Module):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.model = model
        self.clf = nn.Sequential(
            nn.SiLU(),
            nn.LazyLinear(25),
        )

    def forward(self, x, label=None):
        x = self.model(x)
        logits = self.clf(x)
        loss = None
        if label is not None:
            loss = nn.CrossEntropyLoss()(logits, label)
        probs = nn.LogSoftmax(dim=-1)(logits)
        return probs, loss

class LitCustomModel(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = CustomModel(model)
        self.validation_step_output = []

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return opt

    def training_step(self, batch, batch_idx=None):
        x = batch['pixel_values']
        label = batch['label']
        probs, loss = self.model(x, label)
        self.log(f"train_loss", loss, on_step=True, on_epoch=False)
        return loss

    def validation_step(self, batch, batch_idx=None):
        x = batch['pixel_values']
        label = batch['label']
        probs, loss = self.model(x, label)
        self.validation_step_output.append([probs, label, loss])  # Add loss to validation_step_output
        return loss  # Return loss value

    def predict_step(self, batch, batch_idx=None):
        x = batch['pixel_values']
        probs, _ = self.model(x)
        return probs

    def validation_epoch_end(self, step_output):
        pred = torch.cat([x for x, _, _ in self.validation_step_output]).cpu().detach().numpy().argmax(1)
        label = torch.cat([label for _, label, _ in self.validation_step_output]).cpu().detach().numpy()
        score = f1_score(label, pred, average='macro')

        # Calculate validation loss
        val_loss = torch.stack([loss for _, _, loss in self.validation_step_output]).mean()

        self.log("val_score", score)
        self.log("val_loss", val_loss)  # Log validation loss
        self.validation_step_output.clear()
        return score


In [ ]:
#라벨 스무딩 추가
'''class LitCustomModel(L.LightningModule):
    def __init__(self, model, label_smoothing=0.05):
        super().__init__()
        self.model = CustomModel(model)
        self.label_smoothing = label_smoothing
        self.loss_fn = nn.CrossEntropyLoss(label_smoothing=self.label_smoothing)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-5)

    def training_step(self, batch, batch_idx):
        x = batch['pixel_values']
        label = batch['label']
        logits = self.model(x)  # Logits 출력
        loss = self.loss_fn(logits, label)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x = batch['pixel_values']
        label = batch['label']
        logits = self.model(x)  # Logits 출력
        loss = self.loss_fn(logits, label)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        preds = torch.argmax(logits, dim=1)
        return {"loss": loss, "preds": preds, "labels": label}

    def validation_epoch_end(self, outputs):
        preds = torch.cat([x["preds"] for x in outputs], dim=0)
        labels = torch.cat([x["labels"] for x in outputs], dim=0)
        val_loss = torch.stack([x["loss"] for x in outputs]).mean()
        val_score = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average="macro")
        self.log("val_score", val_score, prog_bar=True)
        return val_loss'''

'class LitCustomModel(L.LightningModule):\n    def __init__(self, model, label_smoothing=0.05):\n        super().__init__()\n        self.model = CustomModel(model)\n        self.label_smoothing = label_smoothing\n        self.loss_fn = nn.CrossEntropyLoss(label_smoothing=self.label_smoothing)\n\n    def configure_optimizers(self):\n        return torch.optim.AdamW(self.parameters(), lr=1e-5)\n\n    def training_step(self, batch, batch_idx):\n        x = batch[\'pixel_values\']\n        label = batch[\'label\']\n        logits = self.model(x)  # Logits 출력\n        loss = self.loss_fn(logits, label)\n        self.log("train_loss", loss, on_step=True, on_epoch=True)\n        return loss\n\n    def validation_step(self, batch, batch_idx):\n        x = batch[\'pixel_values\']\n        label = batch[\'label\']\n        logits = self.model(x)  # Logits 출력\n        loss = self.loss_fn(logits, label)\n        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)\n        pred

In [ ]:
SEED = 7
N_SPLIT = 5
BATCH_SIZE = 32

In [ ]:
L.seed_everything(SEED)

INFO:pytorch_lightning.utilities.seed:Global seed set to 7


7

In [ ]:
def dataset_extract(file_name, file_path) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if not os.path.exists(file_path) :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=file_path)

In [ ]:
dataset_extract('/content/drive/MyDrive/조류해상도/open.zip', './')

Extracting:   0%|          | 0/38460 [00:00<?, ?files/s]

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/조류해상도/train.csv')
train_df.head()

,img_path,upscale_img_path,label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/조류해상도/train.csv')
le = LabelEncoder()
train_df['class'] = le.fit_transform(train_df['label'])

In [ ]:
if not len(train_df) == len(os.listdir('./train')):
    raise ValueError()

In [ ]:
skf = StratifiedKFold(n_splits=N_SPLIT, random_state=SEED, shuffle=True)

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(size=(196,196), interpolation=transforms.InterpolationMode.BICUBIC),
    RandomAffine(degrees=30, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    RandomHorizontalFlip(p=0.5),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

val_transform = transforms.Compose([
    transforms.Resize(size=(196,196), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

train_collate_fn = CustomCollateFn(train_transform, 'train')
val_collate_fn = CustomCollateFn(val_transform, 'val')

- 아래 학습하는 코드는 3명이 학습한 모델을 앙상블 했기 때문에, for _ in range(3):을 써서 3번 학습했습니다.
- 따라서, 한번만 모델을 돌리고 싶은 경우 첫번째 for문을 빼주시면 됩니다!

In [ ]:
for _ in range(3):
    for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
        train_fold_df = train_df.loc[train_index,:]
        val_fold_df = train_df.loc[val_index,:]

        train_dataset = CustomDataset(train_fold_df, 'img_path', mode='train')
        val_dataset = CustomDataset(val_fold_df, 'img_path', mode='val')

        train_dataloader = DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=BATCH_SIZE)
        val_dataloader = DataLoader(val_dataset, collate_fn=val_collate_fn, batch_size=BATCH_SIZE*2)

        model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True)
        lit_model = LitCustomModel(model)

        checkpoint_callback = ModelCheckpoint(
            monitor='val_score',
            mode='max',
            dirpath='./checkpoints/',
            filename=f'eva-large-196-resize-fold_idx={fold_idx}'+'-{epoch:02d}-{train_loss:.4f}-{val_loss:.4f}-{val_score:.4f}',
            save_top_k=1,
            save_weights_only=True,
            verbose=True
        )

        earlystopping_callback = EarlyStopping(monitor="val_score", mode="max", patience=6)
        trainer = L.Trainer(max_epochs=10, accelerator='gpu', precision=32, callbacks=[checkpoint_callback, earlystopping_callback], val_check_interval=0.5)
        trainer.fit(lit_model, train_dataloader, val_dataloader)

        model.cpu()
        lit_model.cpu()
        del model, lit_model, checkpoint_callback, earlystopping_callback, trainer
        gc.collect()
        torch.cuda.empty_cache()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory ./checkpoints/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type        | Params
--------------------------------------
0 | model | CustomModel | 304 M 
--------------------------------------
304 M     Trainable params
0         Non-trainable params
304 M     Total params
1,216.569 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 396: 'val_score' reached 0.97245 (best 0.97245), saving model to './checkpoints/eva-large-196-resize-fold_idx=0-epoch=00-train_loss=0.1330-val_loss=0.1149-val_score=0.9725.ckpt' as top 1
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory ./checkpoints/ exists and is not empty.
  rank_zero_warn(f"Chec

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


KeyboardInterrupt: 

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/조류해상도/test.csv')

In [ ]:
if not len(test_df) == len(os.listdir('./test')):
    raise ValueError()

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(size=(196,196), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

test_collate_fn = CustomCollateFn(test_transform, 'inference')
test_dataset = CustomDataset(test_df, 'img_path', mode='inference')
test_dataloader = DataLoader(test_dataset, collate_fn=test_collate_fn, batch_size=BATCH_SIZE*2)


In [ ]:


# 소프트보팅을 위한 예측
fold_probs = []
for checkpoint_path in glob('/content/checkpoints/*.ckpt'):
    model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True)
    lit_model = LitCustomModel.load_from_checkpoint(checkpoint_path, model=model)
    trainer = L.Trainer( accelerator='auto', precision=32)
    preds = trainer.predict(lit_model, test_dataloader)
    probs = F.softmax(torch.cat(preds, dim=0), dim=1).detach().cpu().numpy()  # 각 클래스에 대한 확률값으로 변환
    fold_probs.append(probs)




/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/cloud_io.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locat

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/조류해상도/sample_submission.csv')

In [ ]:
# 소프트보팅
# 각 모델에서 예측한 확률을 평균화하기 위해 모든 예측 확률을 쌓습니다.
pred_probs = np.stack(fold_probs, axis=0)

# 각 클래스에 대한 평균 예측 확률을 계산합니다.
mean_probs = np.mean(pred_probs, axis=0)

# 각 샘플에 대해 가장 높은 평균 확률을 가진 클래스를 선택합니다.
pred_ensemble_soft = np.argmax(mean_probs, axis=1)

submission['label'] = le.inverse_transform(pred_ensemble_soft)

In [ ]:
submission.to_csv('/content/drive/MyDrive/submission/eva_large_196_silu_softvoting_submit.csv',index=False)

# 제출한 모델 재현
- 아래 코드는 저희가 최종 제출한 모델의 가중치를 다운로드, 압축 해제해서 예측한 코드입니다.
- 최종 Private 점수 재현은 아래 코드로 해주시면 감사하겠습니다.

## 체크포인트 압축파일 다운로드 및 해제
## 압축파일 다운로드

In [ ]:
import gdown
import glob

In [ ]:
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [ ]:
checkpoint_file_list = ['1XxX5e2shpxIQt6tnrXaBxdP4ue4HfcfE', '10abqWneGvnpyc9ZWhkw8a2svcdhTKWwI', '1Ho4KpxpHLGKN3yRDYih6wcEPLuRZWmu8']
for i, checkpoint in zip(range(1,len(checkpoint_file_list)+1), checkpoint_file_list):
    download_file(checkpoint, f'checkpoint_{i}.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1XxX5e2shpxIQt6tnrXaBxdP4ue4HfcfE
From (redirected): https://drive.google.com/uc?id=1XxX5e2shpxIQt6tnrXaBxdP4ue4HfcfE&confirm=t&uuid=c105619f-3748-413b-b4da-8c10473fc509
To: /content/checkpoint_1.zip

  0%|          | 0.00/5.66G [00:00<?, ?B/s]
  0%|          | 8.91M/5.66G [00:00<01:25, 66.2MB/s]
  0%|          | 23.6M/5.66G [00:00<00:52, 107MB/s] 
  1%|          | 35.1M/5.66G [00:00<00:50, 111MB/s]
  1%|          | 46.7M/5.66G [00:00<00:52, 106MB/s]
  1%|          | 60.3M/5.66G [00:00<00:48, 115MB/s]
  1%|▏         | 78.1M/5.66G [00:00<00:41, 135MB/s]
  2%|▏         | 92.3M/5.66G [00:00<00:44, 124MB/s]
  2%|▏         | 105M/5.66G [00:00<00:44, 125MB/s] 
  2%|▏         | 118M/5.66G [00:01<00:45, 123MB/s]
  2%|▏         | 135M/5.66G [00:01<01:06, 83.0MB/s]
  3%|▎         | 151M/5.66G [00:01<00:56, 98.3MB/s]
  3%|▎         | 168M/5.66G [00:01<00:47, 115MB/s] 
  3%|▎         | 185M/5.66G [00:01<00:47, 116MB/s]
  4%|▎         |

## 압축파일 해제

In [56]:
import subprocess

def extract_archive(file_name, output_path):
    os.makedirs(output_path, exist_ok=True)
    subprocess.run(['unzip', '-o', file_name, '-d', output_path])

for zip_path in glob.glob('/content/*.zip'):
    print(f'Extracting: {zip_path}')
    extract_archive(zip_path, './')

Extracting: /content/checkpoint_3.zip
Extracting: /content/checkpoint_1.zip
Extracting: /content/checkpoint_2.zip


## 소프트보팅

In [57]:
# 소프트보팅을 위한 예측
fold_probs = []
for checkpoint_path in glob.glob('/content/*ckpt'):
    model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True)
    lit_model = LitCustomModel.load_from_checkpoint(checkpoint_path, model=model)
    trainer = L.Trainer( accelerator='auto', precision=32)
    preds = trainer.predict(lit_model, test_dataloader)
    probs = F.softmax(torch.cat(preds, dim=0), dim=1).detach().cpu().numpy()  # 각 클래스에 대한 확률값으로 변환
    fold_probs.append(probs)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/cloud_io.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locat

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

## 제출파일 불러오기

In [58]:
submission = pd.read_csv('./sample_submission.csv')

In [59]:
# 소프트보팅
# 각 모델에서 예측한 확률을 평균화하기 위해 모든 예측 확률을 쌓습니다.
pred_probs = np.stack(fold_probs, axis=0)

# 각 클래스에 대한 평균 예측 확률을 계산합니다.
mean_probs = np.mean(pred_probs, axis=0)

# 각 샘플에 대해 가장 높은 평균 확률을 가진 클래스를 선택합니다.
pred_ensemble_soft = np.argmax(mean_probs, axis=1)

submission['label'] = le.inverse_transform(pred_ensemble_soft)

In [60]:
#submission.to_csv('/content/drive/MyDrive/AIVLE/dacon/eva_large_196_silu/eva_large_196_silu_softvoting_submit_final.csv',index=False)